# Deep Dive into Langchain

In [2]:
msg = "Welcome to learn Python_Langchain"
print(msg)
print("This is a test for Langchain")

Welcome to learn Python_Langchain
This is a test for Langchain


In [3]:
pip show langchain # to check whether langchain is installed

Name: langchain
Version: 0.3.23
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: c:\Users\abhis\Desktop\AIAgents\LLMAppStreamlit\env\Lib\site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 
---
Name: langchain
Version: 0.3.23
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: c:\Users\abhis\Desktop\AIAgents\LLMAppStreamlit\env\Lib\site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip show langchain-google-genai

Name: langchain-google-genai
Version: 2.1.2
Summary: An integration package connecting Google's genai package and LangChain
Home-page: https://github.com/langchain-ai/langchain-google
Author: 
Author-email: 
License: MIT
Location: c:\Users\abhis\Desktop\AIAgents\LLMAppStreamlit\env\Lib\site-packages
Requires: filetype, google-ai-generativelanguage, langchain-core, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


### Python-dotenv
Python-dotenv is a python module that allows you to specify environment variables as key value pairs in a .env file within your python project directory.It is a convinient and secure way to load and use environment variables in your application.We can have more than one API key and will save them all in .env.
- In this project, we will have an API key for Google's GEMINI, another one for pinecone , Hugging Face and so on.
- create an API Key to access google GEMINI models - [go here](https://aistudio.google.com/app/apikey) & copy the API KEY and save it to our .env file , which we will create.
- let's load environment varaibles.

In [5]:
import os
from dotenv import load_dotenv , find_dotenv
# load the .env file
load_dotenv(find_dotenv(),override=True)
# to get the value of an environment variable
# os.environ.get('GOOGLE_API_KEY') - display the key

True

- We know that langchain provides a standard interface for interacting with various LLMs, including openAI's GPT models , Google's GEMINI, Meta's LLAMA ..etc
- In Langchain terminology these are called LLM providers.
- `Let's see how to invoke the models like Google GEMINI.`
- The models expose an interface where chat messages or conversations serve as inputs & outputs
- refer this [documentation](https://ai.google.dev/gemini-api/docs) to understand more on what to use and how to use like chat-completions.
- [Multi-turn conversation](https://ai.google.dev/gemini-api/docs/text-generation#multi-turn-conversations) & [system instructions](https://ai.google.dev/gemini-api/docs/text-generation#system-instructions) - System instructions let you steer the behavior of a model based on your specific use case. When you provide system instructions, you give the model additional context to help it understand the task and generate more customized responses. The model should adhere to the system instructions over the full interaction with the user, enabling you to specify product-level behavior separate from the prompts provided by end users.
- from langchain we use [messages](https://python.langchain.com/api_reference/core/messages.html) - Messages are objects used in prompts and chat conversations.
- like below:
```py
from langchain_core.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(
        content="You are a helpful assistant! Your name is Bob."
    ),
    HumanMessage(
        content="What is your name?"
    )
]
# Instantiate a chat model and invoke it with the messages
model = ...
print(model.invoke(messages))
```

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
output = llm.invoke("explain virat kohli's batting style in one sentence")
print(output.content)

Virat Kohli's batting style is characterized by his aggressive, yet technically sound approach, combining powerful strokeplay with exceptional running between the wickets and a relentless pursuit of excellence.


In [11]:
# help(ChatGoogleGenerativeAI) 
# - gives whole documentation stuff on this library 
# like Help on class ChatGoogleGenerativeAI in module langchain_google_genai.chat_models: ....etc
# we can see the default models and temparature and other parameters in the documentation is set to

In [15]:
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage
messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="What is the capital of India?")
]
llm.invoke(messages)
# print(assistant_response)

AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-94872e41-a6b2-4a51-b099-aaf00334c829-0', usage_metadata={'input_tokens': 12, 'output_tokens': 9, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}})

- so above we can see that we're passing how the LLM to behave by using Messages(AI,System,Human) to let llm understand what it needs to do and respond accordingly.
- [SystemMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.system.SystemMessage.html) - Message for priming AI behavior. The system message is usually passed in as the first of a sequence of input messages.`Pass in content as positional arg.`
- [HumanMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html) - Message from a human. HumanMessages are messages that are passed in from a human to the model.
- [AIMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html) - Message from an AI.`AIMessage is returned from a chat model as a response to a prompt`.This message represents the output of the model and consists of both the raw output as returned by the model together standardized fields (e.g., tool calls, usage metadata) added by the LangChain framework.
- this is how the `actual` assistant_response from above looks like which returns the AIMessage.
```py
AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-94872e41-a6b2-4a51-b099-aaf00334c829-0', usage_metadata={'input_tokens': 12, 'output_tokens': 9, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}})

```

### Caching in LangChain
- we use caching in langchain to boost performance & save costs.
    - Caching is the practice of storing frequently accessed data or results in a temporary, faster storage layer.
    - In the context of LLM, Caching optimizes interactions with LLMs by reducing API calls and speeding up applications, resulting in a more efficient user experience.When you repeatedly request the same completion from an LLM, caching ensures that the result is stored locally.Subsequent requests for the same input can then be served directly from the cache, reducing the number of expensive API calls to the LLM provider.
    - By avoiding redundant API calls, caching significantly speeds up your application, whether you are building chatbots,content generators or any other language related tools, faster-responses enhance the user experience.
    - let's see how it's done in [langchain](https://python.langchain.com/api_reference/core/caches.html).which provides two options - 1-[in-memory cache](https://python.langchain.com/api_reference/core/caches/langchain_core.caches.InMemoryCache.html#langchain_core.caches.InMemoryCache) & 2-SQLlite-cache - [cache api reference](https://python.langchain.com/api_reference/langchain/globals.html) & we can find related examples in the official documentation on how to use this - [in our case](https://python.langchain.com/docs/how_to/llm_caching/)

#### In-Memory Cache

In [16]:
from langchain.globals import set_llm_cache
from langchain_google_genai import ChatGoogleGenerativeAI
llm_cache=ChatGoogleGenerativeAI(model="gemini-2.0-flash")


In [18]:
%%time 
# to check the time taken to run the code
from langchain_core.caches import InMemoryCache #Cache that stores things in memory.
set_llm_cache(InMemoryCache()) #set the cache to be in memory
prompt= "tell me a joke about python & javascript"
llm_cache.invoke(prompt)

CPU times: total: 0 ns
Wall time: 2.47 s


AIMessage(content='Why did the JavaScript developer quit their job and start learning Python?\n\nBecause they heard Python was great for "Data Handling" and they were tired of always having to deal with "Undefined."', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-ae512894-2ea9-4c9b-8a4a-07aae309f658-0', usage_metadata={'input_tokens': 8, 'output_tokens': 39, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}})

In [20]:
%%time
# let's make the same request again to check the memory cache
llm_cache.invoke(prompt) # this time it will be faster as it is in memory cache

CPU times: total: 0 ns
Wall time: 6.75 ms


AIMessage(content='Why did the JavaScript developer quit their job and start learning Python?\n\nBecause they heard Python was great for "Data Handling" and they were tired of always having to deal with "Undefined."', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-ae512894-2ea9-4c9b-8a4a-07aae309f658-0', usage_metadata={'input_tokens': 8, 'output_tokens': 39, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}})

- Above we can see the difference in time taken to run the code & how caching works to speed up the process.
- let's see [SQLite](https://python.langchain.com/docs/how_to/llm_caching/#sqlite-cache) Caching

In [24]:
%%time
# We can do the same thing with a SQLite cache
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))
# First request(not in cache, takes longer)
llm_cache.invoke("Tell me about virat kohli")

CPU times: total: 15.6 ms
Wall time: 5.47 s


AIMessage(content="Virat Kohli is an Indian international cricketer and former captain of the Indian national cricket team. He is widely regarded as one of the greatest batsmen of all time. Here's a breakdown of his key aspects:\n\n**Key Facts and Achievements:**\n\n*   **Full Name:** Virat Kohli\n*   **Nickname:** Cheeku\n*   **Date of Birth:** November 5, 1988\n*   **Place of Birth:** Delhi, India\n*   **Batting Style:** Right-handed\n*   **Bowling Style:** Right-arm medium\n*   **Role:** Top-order batsman\n*   **Teams:** India, Royal Challengers Bangalore (IPL), Delhi (Domestic)\n*   **Captaincy:** Former captain of the Indian national team across all formats.\n*   **ICC Rankings:** Consistently ranked among the top batsmen in the world.\n\n**Major Achievements and Records:**\n\n*   **Most Runs in T20 Internationals:** He is the leading run-scorer in T20 International cricket.\n*   **Most Centuries for India:** He has the second-most international centuries overall, behind Sachin Te

In [26]:
%%time
# Second request(in cache, takes no time)
llm_cache.invoke("Tell me about virat kohli")

CPU times: total: 0 ns
Wall time: 1.35 ms


AIMessage(content="Virat Kohli is an Indian international cricketer and former captain of the Indian national cricket team. He is widely regarded as one of the greatest batsmen of all time. Here's a breakdown of his key aspects:\n\n**Key Facts and Achievements:**\n\n*   **Full Name:** Virat Kohli\n*   **Nickname:** Cheeku\n*   **Date of Birth:** November 5, 1988\n*   **Place of Birth:** Delhi, India\n*   **Batting Style:** Right-handed\n*   **Bowling Style:** Right-arm medium\n*   **Role:** Top-order batsman\n*   **Teams:** India, Royal Challengers Bangalore (IPL), Delhi (Domestic)\n*   **Captaincy:** Former captain of the Indian national team across all formats.\n*   **ICC Rankings:** Consistently ranked among the top batsmen in the world.\n\n**Major Achievements and Records:**\n\n*   **Most Runs in T20 Internationals:** He is the leading run-scorer in T20 International cricket.\n*   **Most Centuries for India:** He has the second-most international centuries overall, behind Sachin Te